In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import fastai
from fastai.vision import *
from fastai.callbacks import SaveModelCallback
import os
import sys
sys.path.insert(0, './fastai_utils/')
from radam import *
from csvlogger import *
from mish_activation import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
print(fastai.__version__)
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))
    print('  allocated:', round(torch.cuda.memory_allocated(i) / 1024 ** 3, 1), 'GB')
    print('  cached:   ', round(torch.cuda.memory_cached(i) / 1024 ** 3, 1), 'GB')

1.0.61
0 Graphics Device
  allocated: 0.0 GB
  cached:    0.0 GB
1 Graphics Device
  allocated: 0.0 GB
  cached:    0.0 GB


In [2]:
sz = 256
bs = 8
nfolds = 4
SEED = 80
N_WORKERS = 8
N = 20
MODEL_VER = 'v14'
MAIN_PATH = '.'
DATA_PATH = '{}/data'.format(MAIN_PATH)
TRAIN = '{}/train_images_tiles_q1_256_36/'.format(DATA_PATH)
LABELS = '{}/train.csv'.format(DATA_PATH)
MODELS = '{}/fastai_models'.format(MAIN_PATH)
if not os.path.exists('{}_{}'.format(MODELS, MODEL_VER)):
    os.mkdir('{}_{}'.format(MODELS, MODEL_VER))
    print('created:', '{}_{}'.format(MODELS, MODEL_VER))
# v_ = sz 256 bs 6 N 12 folds 4 --> 0.8398 (1st fold)
# v0 = sz 256 bs 8 N 8  folds 4 --> 0.8057 (1st fold)
# v1 = sz 512 bs 6 N 3 folds 4 --> 0.7959 (1st fold)
# v2 = sz 256 bs 5 N 14 folds 4 max_lr 1e-3 df 1000 --> .77998 (1st fold)
# v3 = sz 256 bs 6 N 12 folds 4 max_lr 1e-3 df 1000 --> 
# v4 = sz 256_36 bs 6 N 12 folds 4 max_lr 3e-4 df 100 --> .8382(0), .8234(1), .8392(2), .8251(2), .8314(all) --> .84 LB
# v5 = sz 256_36 bs 6 N 12 folds 4 max_lr 5e-4 df 500 ep 40 --> .8341 (1st fold)
# v6 = sz 128_24_r2 bs 12 N 24 folds 4 max_lr 3e-4 df 100 ep 30 --> .8400 (1st fold)
# v7 = sz 128_32 bs 8 N 32 folds 4 max_lr 3e-4 df 100 ep 30 --> .8422(0) .8380(1) .8423(2) .8340(3) .8391(all) --> .85 LB
# v8 = sz 256_36 bs 12(x2) N 12 folds 4 max_lr 3e-4 df 100 --> .8400(0)
# v9 = sz 256_36 bs 6(x2) N 22 folds 4 max_lr 3e-4 df 100 ep 30 --> .8264(0)
# v10 = sz 256_36 bs 8(x2) N 18 folds 4 max_lr 3e-4 df 100 ep 30 --> .8510(0)
# v11 = sz 156_64 bs 8(x2) N 48 folds 4 max_lr 3e-4 df 100 ep 30 --> .8527(0)
# v12 = sz 156_64 bs 12(x2) N 36 folds 4 max_lr 3e-4 df 100 ep 30 --> .8423(0)
# v13 = sz 156_64 bs 12(x2) N 54 folds 4 max_lr 3e-4 df 100 ep 40 --> .8493(0)
# v14 =  sz 256_36 bs 8(x2) N 20 folds 4 max_lr 3e-4 df 100 ep 40--> .8626(0) .8513(1) .8539(2) .8547(3) .8556(all)
print('256x256x18 img size:', 256 * 256 * 18)
print('128x128x32 img size:', 128 * 128 * 32)
print('current img size:', sz * sz * N)

256x256x18 img size: 1179648
128x128x32 img size: 524288
current img size: 1310720


In [3]:
df = pd.read_csv(LABELS).set_index('image_id')
files = sorted(set([p[:32] for p in os.listdir(TRAIN) if '.ipynb' not in p]))
df = df.loc[files]
df = df.reset_index()
splits = StratifiedKFold(n_splits=nfolds, random_state=SEED, shuffle=True)
splits = list(splits.split(df,df.isup_grade))
folds_splits = np.zeros(len(df)).astype(np.int)
for i in range(nfolds): folds_splits[splits[i][1]] = i
df['split'] = folds_splits
df.head()

,image_id,data_provider,isup_grade,gleason_score,split
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,1
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,2
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,0
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,3
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,2


In [4]:
# q1_256_36
mean = torch.tensor([1 - .85506157, 1 - .7035249, 1 - .80203127])
std = torch.tensor([.40011922, .52504386, .42675745])
# q1_512_16
#mean = torch.tensor([1 - .91066496, 1 - .82012713, 1 - .87912669])
#std = torch.tensor([.37697129, .50629405, .41176592])
# q1_256_24
#mean = torch.tensor([1 - .82934477, 1 - .64875879, 1 - .76593138])
#std = torch.tensor([.40447862, .51985678, .42330205])
# q1_128_32
#mean = torch.tensor([1 - .75793065, 1 - .5088926, 1 - .67191824])
#std = torch.tensor([.41296871, .47099853, .39308129])
# q1_128_24_r2
#mean = torch.tensor([1 - .8296201, 1 - .64903966, 1 - .76621777])
#std = torch.tensor([.39454631, .51460749, .41747009])
# q1_156_64
#mean = torch.tensor([1 - .81198481, 1 - .61055975, 1 - .74127657])
#std = torch.tensor([.40365266, .50938881, .41515295])

In [5]:
def open_image(fn:PathOrStr, div:bool=True, convert_mode:str='RGB', cls:type=Image,
        after_open:Callable=None)->Image:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning) # EXIF warning from TiffPlugin
        x = PIL.Image.open(fn).convert(convert_mode)
    if after_open: x = after_open(x)
    x = pil2tensor(x,np.float32)
    if div: x.div_(255)
    return cls(1.0-x) #invert image for zero padding
class MImage(ItemBase):
    def __init__(self, imgs):
        self.obj, self.data = \
          (imgs), [(imgs[i].data - mean[...,None,None])/std[...,None,None] for i in range(len(imgs))]
    def apply_tfms(self, tfms,*args, **kwargs):
        for i in range(len(self.obj)):
            self.obj[i] = self.obj[i].apply_tfms(tfms, *args, **kwargs)
            self.data[i] = (self.obj[i].data - mean[...,None,None])/std[...,None,None]
        return self
    def __repr__(self): return f'{self.__class__.__name__} {img.shape for img in self.obj}'
    def to_one(self):
        img = torch.stack(self.data,1)
        img = img.view(3,-1,N,sz,sz).permute(0,1,3,2,4).contiguous().view(3,-1,sz*N)
        return Image(1.0 - (mean[...,None,None]+img*std[...,None,None]))
class MImageItemList(ImageList):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def __len__(self)->int: return len(self.items) or 1 
    def get(self, i):
        fn = Path(self.items[i])
        fnames = [Path(str(fn)+'_'+str(i)+'.png')for i in range(N)]
        imgs = [open_image(fname, convert_mode=self.convert_mode, after_open=self.after_open)
               for fname in fnames]
        return MImage(imgs)
    def reconstruct(self, t):
        return MImage([mean[...,None,None]+_t*std[...,None,None] for _t in t])
    def show_xys(self, xs, ys, figsize:Tuple[int,int]=(300,50), **kwargs):
        rows = min(len(xs),8)
        fig, axs = plt.subplots(rows,1,figsize=figsize)
        for i, ax in enumerate(axs.flatten() if rows > 1 else [axs]):
            xs[i].to_one().show(ax=ax, y=ys[i], **kwargs)
        plt.tight_layout()
def MImage_collate(batch:ItemsList)->Tensor:
    result = torch.utils.data.dataloader.default_collate(to_data(batch))
    if isinstance(result[0],list):
        result = [torch.stack(result[0],1),result[1]]
    return result

In [6]:
def get_data(fold=0):
    return (
        MImageItemList.from_df(df, path='.', folder=TRAIN, cols='image_id')
        .split_by_idx(df.index[df.split == fold].tolist())
        .label_from_df(cols=['isup_grade'])
        .transform(
            get_transforms(
                flip_vert=True,
                max_rotate=15,
                max_warp=None
            ),
            size=sz,
            padding_mode='zeros'
        ).databunch(
            bs=bs, 
            num_workers=N_WORKERS
        )
    )
data = get_data(0)
#data.show_batch()

In [7]:
class Model(nn.Module):
    def __init__(self, arch='resnext50_32x4d_ssl', n=6, pre=True):
        super().__init__()
        m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', arch)
        self.enc = nn.DataParallel(nn.Sequential(*list(m.children())[:-2]))
        nc = list(m.children())[-1].in_features
        self.head = nn.DataParallel(nn.Sequential(
            AdaptiveConcatPool2d(),
            Flatten(),
            nn.Linear(2 * nc, 512),
            Mish(),
            nn.BatchNorm1d(512), 
            nn.Dropout(.4),
            nn.Linear(512, n)
        ))
    def forward(self, *x):
        shape = x[0].shape
        n = len(x)
        x = torch.stack(x,1).view(
            -1,
            shape[1],
            shape[2],
            shape[3]
        )
        #x: bs*N x 3 x 128 x 128
        x = self.enc(x)
        #x: bs*N x C x 4 x 4
        shape = x.shape
        #concatenate the output for tiles into a single map
        x = x.view(
            -1,
            n,
            shape[1],
            shape[2],
            shape[3]
        ).permute(0, 2, 1, 3, 4).contiguous().view(
            -1,
            shape[1],
            shape[2] * n,
            shape[3]
        )
        #x: bs x C x N*4 x 4
        x = self.head(x)
        #x: bs x n
        return x

In [8]:
fname = 'RNXT50'
pred, target = [], []
NFOLDS = nfolds
for fold in range(NFOLDS):
    print('-' * 20, 'fold:', fold, '-' * 20)
    data = get_data(fold)
    model = Model()
    learn = Learner(
        data, 
        model, 
        loss_func=nn.CrossEntropyLoss(), 
        opt_func=Over9000, 
        metrics=[KappaScore(weights='quadratic')]
    ).to_fp16()
    logger = CSVLogger(learn, '{}_{}/log_{}_{}'.format(MODELS, MODEL_VER, fname, fold))
    learn.clip_grad = 1.0
    learn.split([model.head])
    learn.unfreeze()
    learn.fit_one_cycle(
        40, 
        max_lr=3e-4,
        div_factor=100, 
        pct_start=0.0, 
        callbacks = [
            SaveModelCallback(
                learn, 
                name='model_{}'.format(MODEL_VER), 
                monitor='kappa_score'
            )
        ]
    )
    torch.save(learn.model.state_dict(), '{}_{}/{}_fold{}.pth'.format(MODELS, MODEL_VER, fname, fold))
    learn.model.eval()
    pred_fold, target_fold = [], []
    with torch.no_grad():
        for step, (x, y) in progress_bar(
            enumerate(data.dl(DatasetType.Valid)),
            total=len(data.dl(DatasetType.Valid))
        ):
            p = learn.model(*x)
            pred_step = p.float().cpu()
            target_step = y.cpu()
            pred.append(pred_step)
            target.append(target_step)
            pred_fold.append(pred_step)
            target_fold.append(target_step)
    p = torch.argmax(torch.cat(pred_fold,0),1)
    t = torch.cat(target_fold)
    print(cohen_kappa_score(t,p,weights='quadratic'))
    print(confusion_matrix(t,p))

-------------------- fold: 0 --------------------


Using cache found in /u01/mrorange/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master


epoch,train_loss,valid_loss,kappa_score,time
0,1.349970,1.123420,0.723867,16:51
1,1.400526,1.087826,0.745991,16:47
2,1.349426,1.224559,0.759278,16:46
3,1.252211,1.053766,0.682915,16:46
4,1.183372,1.224531,0.626279,16:46
5,1.150523,0.957378,0.791645,16:46
6,1.178163,0.964952,0.795631,16:47
7,1.201370,1.052269,0.623135,16:47
8,1.182096,1.051901,0.735533,16:47
9,1.110160,1.068828,0.770932,16:46


Better model found at epoch 0 with kappa_score value: 0.7238669395446777.
Better model found at epoch 1 with kappa_score value: 0.7459914684295654.
Better model found at epoch 2 with kappa_score value: 0.7592782974243164.
Better model found at epoch 5 with kappa_score value: 0.791645348072052.
Better model found at epoch 6 with kappa_score value: 0.7956308126449585.
Better model found at epoch 11 with kappa_score value: 0.7970012426376343.
Better model found at epoch 12 with kappa_score value: 0.8063660264015198.
Better model found at epoch 17 with kappa_score value: 0.8254194259643555.
Better model found at epoch 18 with kappa_score value: 0.8257757425308228.
Better model found at epoch 23 with kappa_score value: 0.8308830261230469.
Better model found at epoch 25 with kappa_score value: 0.8408898115158081.
Better model found at epoch 27 with kappa_score value: 0.8469771146774292.
Better model found at epoch 31 with kappa_score value: 0.8559314012527466.
Better model found at epoch 32 

0.8626681978895994
[[685  29   2   0   3   0]
 [ 62 528  47   6  10   1]
 [  9 103 145  59  17   2]
 [ 21  15  29 145  69  27]
 [ 19  11   8  36 201  36]
 [ 12   4   4  24  81 179]]
-------------------- fold: 1 --------------------


...

0.8539228367504973
[[676  37   1   0   3   1]
 [ 62 522  58   4   7   1]
 [ 15  89 150  61  16   4]
 [ 17  23  37 134  69  27]
 [ 19  11  13  48 180  40]
 [ 15   3   1  29  81 175]]
-------------------- fold: 3 --------------------


Using cache found in /u01/mrorange/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master


epoch,train_loss,valid_loss,kappa_score,time
0,1.491646,1.199876,0.699800,16:45
1,1.296882,1.196307,0.675721,16:45
2,1.302471,1.598838,0.644009,16:45
3,1.242595,1.021652,0.763110,16:45
4,1.235247,1.039450,0.743286,16:45
5,1.232638,0.978703,0.761300,16:45
6,1.107760,1.001514,0.744476,16:45
7,1.132136,1.167315,0.704356,16:45
8,1.194473,1.005017,0.757321,16:45
9,1.158311,0.950148,0.785401,16:45


Better model found at epoch 0 with kappa_score value: 0.6997998952865601.
Better model found at epoch 3 with kappa_score value: 0.7631097435951233.
Better model found at epoch 9 with kappa_score value: 0.7854011654853821.
Better model found at epoch 13 with kappa_score value: 0.7958096861839294.
Better model found at epoch 15 with kappa_score value: 0.8030343055725098.
Better model found at epoch 16 with kappa_score value: 0.8108221888542175.
Better model found at epoch 18 with kappa_score value: 0.8243037462234497.
Better model found at epoch 20 with kappa_score value: 0.837189257144928.
Better model found at epoch 26 with kappa_score value: 0.8398044109344482.
Better model found at epoch 27 with kappa_score value: 0.8425959944725037.
Better model found at epoch 29 with kappa_score value: 0.8433158993721008.
Better model found at epoch 30 with kappa_score value: 0.8500062227249146.
Better model found at epoch 33 with kappa_score value: 0.8547289371490479.


0.8547289735900723
[[679  27   2   0   6   4]
 [ 46 515  71  14   8   0]
 [  7 112 153  41  20   3]
 [ 17  20  42 138  64  26]
 [ 14   6  15  49 199  28]
 [ 15   2   3  25  83 175]]


In [9]:
p = torch.argmax(torch.cat(pred,0),1)
t = torch.cat(target)
print(cohen_kappa_score(t,p,weights='quadratic'))
print(confusion_matrix(t,p))

0.8556796703062655
[[2731  116    5    2   14    5]
 [ 233 2098  228   26   29    2]
 [  40  406  615  198   69   13]
 [  71   77  152  560  256  110]
 [  66   46   49  188  757  139]
 [  64   10   14  110  313  704]]
